# Table of Contents
 <p><div class="lev2 toc-item"><a href="#Enable-IPywidgets-and-Qgrid" data-toc-modified-id="Enable-IPywidgets-and-Qgrid-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Enable IPywidgets and Qgrid</a></div><div class="lev2 toc-item"><a href="#Note" data-toc-modified-id="Note-02"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Note</a></div>

In [ ]:
# monitor training performance  
from numpy import loadtxt  
from xgboost import XGBClassifier,plot_importance  
from xgboost import XGBRegressor,plot_tree
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import accuracy_score  
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import seaborn as sns
sns.set()

Download from:

https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes

In [ ]:
# load data  
df = pd.read_csv('data/pima-indians-diabetes.csv')#, delimiter=",")  

Enable IPywidgets and Qgrid
---
Install as follws:
```
> pip install ipywidgets
> jupyter nbextension enable --py --sys-prefix widgetsnbextension

> pip install qgrid
> jupyter nbextension enable --py --sys-prefix qgrid
```



In [ ]:
import qgrid

In [ ]:
qgrid.show_grid(df)

In [ ]:
features=df.columns.values
features

In [ ]:
# count for Nan
1- df.describe().T['count']/len(df)

In [ ]:
# split data into X and y  
X = df[features[:8]]
Y = df['class']

In [ ]:
# split data into train and test sets  
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=2019)  

- logLoss:  as known as ```logistic loss or cross-entropy loss```, for a single sample with true label $y_t$ in {0,1} and estimated probability $y_p$ that $y_t$ = 1, the log loss is
  $$-\log \mathbf{P} (y_t|y_p) = -\left(y_t \log(y_p) + (1 - y_t) \log(1 - y_p)\right)$$

In [ ]:
# fit model no training data  
model = XGBClassifier()  
eval_set = [(X_train, y_train), (X_test, y_test)]  
#model.fit(X_train, y_train, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=True)  
model.fit(X_train, y_train, eval_metric=["error", "auc"], eval_set=eval_set, verbose=True) 
# make predictions for test data  

In [ ]:
plot_importance(model)

In [ ]:
y_pred=model.predict(X_test) 

In [ ]:
# evaluate predictions  
 
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))  

In [ ]:
# retrieve performance metrics  
results = model.evals_result()  
epochs = len(results['validation_0']['error'])  

In [ ]:
x_axis = range(0, epochs) 

In [ ]:
# plot log loss  
fig, ax = plt.subplots()  
ax.plot(x_axis, results['validation_0']['error'], label='Train')  
ax.plot(x_axis, results['validation_1']['auc'], label='Test')  
ax.legend()  
#plt.ylabel('Log Loss')  
ax.set_title('XGBoost Log Loss')  


In [ ]:
# plot classification error  
fig, ax = plt.subplots()  
ax.plot(x_axis, results['validation_0']['error'], label='Train')  
ax.plot(x_axis, results['validation_1']['error'], label='Test')  
ax.legend()
plt.ylabel('Classification Error')  
plt.title('XGBoost Classification Error')  
#plt.show() 

Note
---
When nround is greater than 40, error of test data arised, this means model is over-fitting. Thus set the stop sign after 10 run:

In [ ]:
# fit model no training data  
xgb = XGBClassifier()  
eval_set = [(X_test, y_test)]  
xgb.fit(X_train, y_train, early_stopping_rounds=10, 
          eval_metric="auc", eval_set=eval_set, verbose=True)  
# make predictions for test data  
y_pred = xgb.predict(X_test)  
#predictions = [round(value) for value in y_pred]  
# evaluate predictions  
accuracy = accuracy_score(y_test, y_pred)  

In [ ]:
# fit model no training data  
xgb = XGBClassifier(ax_depth=5,
                        subsample=0.33,
                        objective='binary:logistic',
                        n_estimators=500,
                        learning_rate = 0.001)  
eval_set = [(X_test, y_test)]  
xgb.fit(X_train, y_train, early_stopping_rounds=10, 
          eval_metric="auc", eval_set=eval_set, verbose=True)  
# make predictions for test data  
y_pred = xgb.predict(X_test)  
#predictions = [round(value) for value in y_pred]  
# evaluate predictions  
accuracy = accuracy_score(y_test, y_pred)  

In [ ]:
print (xgb.best_iteration)  
limit = xgb.best_iteration  
y_pred = xgb.predict(X_test,ntree_limit=limit)  
#predictions = [round(value) for value in y_pred]  
# evaluate predictions  
accuracy = accuracy_score(y_test, y_pred)  
print("Accuracy: %.2f%%" % (accuracy * 100.0))  

In [ ]:
##set up the parameters
plt.rcParams['figure.figsize'] = 80,50
plt.figure(figsize=(12,8))
ax=plot_tree(xgb)  
plt.show()  

In [ ]:
from xgboost import to_graphviz

In [ ]:
import codecs  
f=codecs.open('xgb_tree.png', mode='wb')  
g=to_graphviz(model)  
f.write(g.pipe('png'));  
f.close()  